In [1]:
import cv2
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import time

In [2]:
cd GaitDatasetB-silh

C:\Users\somna\GAN\GaitDatasetB-silh


In [3]:
ls 002\\bg-02

 Volume in drive C is OS
 Volume Serial Number is 181C-9033

 Directory of C:\Users\somna\GAN\GaitDatasetB-silh\002\bg-02

24-02-2006  16:43    <DIR>          .
24-02-2006  16:43    <DIR>          ..
24-02-2006  16:43    <DIR>          000
24-02-2006  16:43    <DIR>          018
24-02-2006  16:43    <DIR>          036
24-02-2006  16:43    <DIR>          054
24-02-2006  16:43    <DIR>          072
24-02-2006  16:43    <DIR>          090
24-02-2006  16:43    <DIR>          108
24-02-2006  16:43    <DIR>          126
24-02-2006  16:43    <DIR>          144
24-02-2006  16:43    <DIR>          162
24-02-2006  16:43    <DIR>          180
               0 File(s)              0 bytes
              13 Dir(s)  12,514,414,592 bytes free


In [4]:
import os

In [51]:
def preprocess(img):
    cnt, heir = cv2.findContours(img[:,:,0],cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    x,y,w,h = cv2.boundingRect(cnt[0])
    temp = img[y:y+h, x:x+w, 0]
    w1 = (h*3)//4
    temp = cv2.copyMakeBorder(temp, 0,0, max(w1-w,0)//2, max(w1-w,0)//2, cv2.BORDER_CONSTANT, (0,0,0))
    return cv2.resize(temp, (120,160))

In [53]:
for file in os.listdir(os.getcwd()+'/001/nm-01/036/'): 
    a = preprocess(cv2.imread('001/nm-01/036/'+file))
    cv2.imshow('frame',a)
    cv2.waitKey(0)
cv2.destroyAllWindows()

In [64]:
def get_feature_vectors(imgs,k=10):
    G = np.vstack(tuple(preprocess(img).reshape(-1).astype(np.float64)/255. for img in imgs))
    avg = G.T.mean(axis=1)
    A = G.T-avg.reshape(-1,1)
    X = A.T@A / A.shape[1]
    eigvalue, eigvec = np.linalg.eigh(X)
    U = A@eigvec
    u = U/np.linalg.norm(U, axis=0)
    u_sorted = u[:,np.argsort(eigvalue)[::-1]]
    u_k = u_sorted[:,0:k]
    W = u_k.T@A
    A1 = u_k@W
    return W, A1, A, u_k, avg

In [65]:
W, A1, A, u_k, avg = get_feature_vectors((cv2.imread('001/bg-02/090/'+file) for file in os.listdir(os.getcwd()+'/001/bg-02/090')))

In [66]:
def assert_images(X1,X, HEIGHT=160, WIDTH=120):
    for i in range(X1.shape[1]):
        cv2.imshow('frame',((X1).reshape(WIDTH, HEIGHT, -1)[:,:,i]))
        cv2.imshow('frame2',((X).reshape(WIDTH, HEIGHT, -1)[:,:,i]))
        cv2.waitKey(0)
    cv2.destroyAllWindows()

In [68]:
assert_images(A1+avg.reshape(-1,1),A+avg.reshape(-1,1),120,160)

In [69]:
class KMeans():
    def __init__(self, K = 16):
        self.K = K
        self.states = [[] for _ in range(self.K)]
        self.P = np.zeros((10,self.K))
    def fit(self,W):
        self.W = W
        for j in range(self.W.shape[1]):
            self.states[int(j*self.K/self.W.shape[1])].append(j)
        for i, state in enumerate(self.states):
            self.P[:,i] = self.W[:,state].mean(axis=1)
        self.converge()
    def converge(self):
        changes = 0
        while changes < 5 :
            prev_states = self.states
            for i, state in enumerate(self.states):
                self.P[:,i] = self.W[:,state].mean(axis=1)
            temp_states = []
            for i in range(self.K):
                temp_state = []
                for j in [max(0,i-1), min(i+1,self.K-1)]:
                    if i == j: continue
                    for state in self.states[j]:
                        if np.linalg.norm(self.W[:,state]-self.P[:,i]) <= np.linalg.norm(self.W[:,state]-self.P[:,j]):
                            print(i,j)
                            temp_state.append([j,state])
                temp_states.append(temp_state)
            for i, temp_state in enumerate(temp_states):
                temp_state.sort(key=lambda x : -x[1] if x[0]<i else x[0])
                for clst, frm in temp_state:
                    if clst<i:
                        if self.states[clst][-1] == frm and len(self.states[clst])>1:
                            self.states[clst].remove(frm)
                        else:
                            temp_states[i].remove([clst,frm])
                    else:
                        if self.states[clst][0] == frm and len(self.states[clst])>1:
                            self.states[clst].remove(frm)
                        else:
                            temp_states[i].remove([clst,frm])
            for i in range(len(self.states)):
                for _, x in temp_states[i]:
                    self.states[i].append(x)
                self.states[i].sort()
            if self.states != prev_states:
                changes = 0
            else:
                changes += 1
    def predict(self, W_t):
        MV = np.array(tuple(np.linalg.norm(W_t - self.P[:,i:i+1],axis=0) for i in range(self.K)))
        MV = 1 - MV / MV.max(axis=0)
        return MV

In [70]:
kmeans = KMeans(K=16)

In [71]:
kmeans.fit(W)
kmeans.states

2 1
3 4
5 4
8 9
11 12
15 14
5 6
9 10
10 11
11 12
12 13


[[0, 1, 2, 3],
 [4, 5, 6],
 [7, 8, 9, 10],
 [11, 12, 13, 14, 15],
 [16, 17],
 [18, 19, 20, 21, 22],
 [23, 24, 25],
 [26, 27, 28],
 [29, 30, 31, 32, 33],
 [34, 35, 36, 37],
 [38, 39, 40],
 [41, 42, 43, 44, 45],
 [46, 47, 48],
 [49, 50],
 [51, 52, 53],
 [54, 55, 56, 57]]

In [75]:
test_imgs = (cv2.imread('001/nm-01/090/'+file) for file in os.listdir(os.getcwd()+'/001/nm-01/090'))

In [76]:
W_t, A1_t , A_t, _, avg_t = get_feature_vectors(test_imgs)

In [78]:
assert_images(A1_t+avg_t.reshape(-1,1), A_t+avg_t.reshape(-1,1), 120, 160)

In [79]:
def graph_sort(k_means, W_i):
    E = k_means.predict(W_i)
    V = np.zeros((E.shape[0]*E.shape[1], E.shape[0]*E.shape[1]))
    decode = lambda x,k : (x%k,x//k)
    encode = lambda i,j,k: j*k + i
    for cnt in range(V.shape[0]-E.shape[0]):
        i,j = decode(cnt, E.shape[0])
        for pos in [(i,j+1),((i+1)%E.shape[0],j+1)]:
            V[cnt,encode(*pos, E.shape[0])] = 1    # if V[x][y] == 1 then x->y only
    PV = np.zeros_like(E)
    for curr in range(V.shape[0]):
        maxx = 0
        for prev in np.where(V[:,curr]==1)[0]:
            temp = PV[decode(prev,E.shape[0])]
            if temp>maxx:
                maxx = temp
        PV[decode(curr,E.shape[0])] = maxx + E[decode(curr,E.shape[0])]
    term_state = (np.argmax(PV[:,-1]),E.shape[1]-1)
    test_states = []
    test_states.append(term_state[0])
    prevs = np.where(V[:,encode(*term_state,E.shape[0])]==1)[0]
    while len(prevs)>0:
        maxx = 0
        pos = 0
        for prev in prevs:
            if maxx < PV[decode(prev,E.shape[0])]:
                maxx = PV[decode(prev,E.shape[0])]
                pos = prev
        term_state = decode(pos,E.shape[0])
        test_states.append(term_state[0])
        prevs = np.where(V[:,encode(*term_state,E.shape[0])]==1)[0]
    test_states = np.asarray(test_states[::-1])
    return test_states

In [80]:
graph_sort(kmeans,W_t)

array([12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 14, 15,  0,  0,  0,
        1,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  4,  5,  5,  5,  5,  5,
        6,  7,  7,  7,  7,  8,  8,  8,  8,  8,  8,  9, 10, 10, 10, 10, 11,
       11, 11, 11, 11, 11], dtype=int64)